## Engenharia de Sistemas de Software Inteligentes - Profs. Marcos Kalinowski e Tatiana Escovedo

In [45]:
# Imports necessários
import pandas as pd
from sklearn.datasets import load_iris # para importar o dataset iris
from sklearn.model_selection import train_test_split # para particionar em bases de treino e teste
from sklearn.metrics import confusion_matrix # para a exibição da matriz de confusão do modelo
from sklearn.metrics import accuracy_score # para a exibição da acurácia do modelo
from sklearn.svm import SVC # para utilizar o algoritmo SVM
# Imports necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import warnings
import lightgbm as lgb
from sklearn.ensemble import AdaBoostRegressor,RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.exceptions import ConvergenceWarning

## Criação de classes

In [67]:
# Em programas mais robustos, teríamos um arquivo .py para cada classe e
# importaríamos as classes para executar o programa:
# from <nome-do-arquivo> import <nome-da-classe>

def categorize_sleep_efficiency(x):
  coeff_sleep = x['Sleep efficiency']
  if coeff_sleep>= 0.5 and coeff_sleep < 0.625:
    x['categoric_sleep_eff'] = 'Pessimo'
  elif coeff_sleep>= 0.625 and coeff_sleep < 0.75:
    x['categoric_sleep_eff'] = 'Ruim'
  elif coeff_sleep>= 0.75 and coeff_sleep < 0.875:
    x['categoric_sleep_eff'] = 'Bom'
  elif coeff_sleep>= 0.75 and coeff_sleep < 1:
    x['categoric_sleep_eff'] = 'Otimo'
  else:
    x['categoric_sleep_eff'] = 'Nan'
  return x


class Carregador:

    def carregar_dados(self, url: str, atributos: list):
        """ Carrega e retorna um DataFrame. Há diversos parâmetros
        no read_csv que poderiam ser utilizados para dar opções
        adicionais.
        """
        return pd.read_csv(url)

class PreProcessador:


    def pre_processar(self, dataset, percentual_teste, seed=7):
        """ Cuida de todo o pré-processamento. """
        # limpeza dos dados
        dataset['sex']=dataset['Gender'].apply(lambda X:1 if X=='Male' else 0)
        dataset['smoking'] = dataset['Smoking status'].apply(lambda x:1 if x == 'Yes' else 0)
        dataset['Bedtime'] = pd.to_datetime(dataset['Bedtime'], format='%Y-%m-%d %H:%M:%S')
        dataset['Wakeup time']=pd.to_datetime(dataset['Wakeup time'],format='%Y-%m-%d %H:%M:%S')
        # Extracting just the hour
        dataset['Bedhour'] = dataset['Bedtime'].dt.hour
        dataset['Wakehour']=dataset['Wakeup time'].dt.hour
        dataset = dataset.apply(categorize_sleep_efficiency,axis = 1)
        dataset.dropna(inplace=True)
        # feature selection
        dataset.drop(['ID','Gender','Bedtime','Wakeup time','Smoking status','Sleep efficiency'],axis='columns', inplace=True)
        # divisão em treino e teste
        X_train, X_test, Y_train, Y_test = self.__preparar_holdout(dataset,
                                                                  percentual_teste,
                                                                  seed)
        # normalização/padronização

        return (dataset,X_train, X_test, Y_train, Y_test)

    def __preparar_holdout(self, dataset, percentual_teste, seed):
        """ Divide os dados em treino e teste usando o método holdout.
        Assume que a variável target está na última coluna.
        O parâmetro test_size é o percentual de dados de teste.
        """
        dados = dataset.values
        X = dados[:, 0:-1]
        Y = dados[:, -1]
        return train_test_split(X, Y, test_size=percentual_teste, random_state=seed)


class Modelo:

    def treinar_SVM(self, X_train, Y_train):
        """ Cria e treina um modelo SVM. Poderia ter um Grid Search
        com cross_validation para escolher os melhores hiperparâmetros, etc.
        """
        # Preparação do modelo
        scaler = StandardScaler().fit(X_train) # ajuste do scaler com o conjunto de treino
        rescaledX = scaler.transform(X_train) # aplicação da padronização no conjunto de treino
        model = GaussianNB()
        model.fit(X_train, Y_train)
        return model


class Avaliador:


    def avaliar_modelos(self,X_train,Y_train):

      np.random.seed(7) # definindo uma semente global para este bloco

      num_particoes = 10
      kfold = StratifiedKFold(n_splits=num_particoes, shuffle=True, random_state=7) # validação cruzada com estratificação
      # Listas para armazenar os armazenar os pipelines e os resultados para todas as visões do dataset
      pipelines = []
      results = []
      names = []


      # Criando os elementos do pipeline

      # Algoritmos que serão utilizados
      knn = ('KNN', KNeighborsClassifier())
      cart = ('CART', DecisionTreeClassifier())
      naive_bayes = ('NB', GaussianNB())
      svm = ('SVM', SVC())

      # Transformações que serão utilizadas
      standard_scaler = ('StandardScaler', StandardScaler())
      min_max_scaler = ('MinMaxScaler', MinMaxScaler())


      # Montando os pipelines

      # Dataset original
      pipelines.append(('KNN-orig', Pipeline([knn])))
      pipelines.append(('CART-orig', Pipeline([cart])))
      pipelines.append(('NB-orig', Pipeline([naive_bayes])))
      pipelines.append(('SVM-orig', Pipeline([svm])))

      # Dataset Padronizado
      pipelines.append(('KNN-padr', Pipeline([standard_scaler, knn])))
      pipelines.append(('CART-padr', Pipeline([standard_scaler, cart])))
      pipelines.append(('NB-padr', Pipeline([standard_scaler, naive_bayes])))
      pipelines.append(('SVM-padr', Pipeline([standard_scaler, svm])))

      # Dataset Normalizado
      pipelines.append(('KNN-norm', Pipeline([min_max_scaler, knn])))
      pipelines.append(('CART-norm', Pipeline([min_max_scaler, cart])))
      pipelines.append(('NB-norm', Pipeline([min_max_scaler, naive_bayes])))
      pipelines.append(('SVM-norm', Pipeline([min_max_scaler, svm])))

      # Executando os pipelines
      for name, model in pipelines:
          cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
          results.append(cv_results)
          names.append(name)
          msg = "%s: %.3f (%.3f)" % (name, cv_results.mean(), cv_results.std()) # formatando para 3 casas decimais
          print(msg)

      # Boxplot de comparação dos modelos
      fig = plt.figure(figsize=(25,6))
      fig.suptitle('Comparação dos Modelos - Dataset orginal, padronizado e normalizado')
      ax = fig.add_subplot(111)
      plt.boxplot(results)
      ax.set_xticklabels(names, rotation=90)
      plt.show()


    def avaliar_acuracia(self, modelo, X_test, Y_test):
        """ Faz uma predição e avalia o modelo. Poderia parametrizar o tipo de
        avaliação, entre outros.
        """
        predicoes = modelo.predict(X_test)
        return accuracy_score(Y_test, predicoes)


# Este código começaria daqui, logo após os imports
# from loader import Loader
# from preprocessor import PreProcessor
# from ml_model import MLModel
# from ml_evaluator import MlEvaluator

# Instanciação das Classes
carregador = Carregador()
pre_processador = PreProcessador()
modelo = Modelo()
avaliador = Avaliador()

# Parâmetros
url_dados = ('https://raw.githubusercontent.com/DDFaller/ML_Sleep_Efficiency/MAIN/data/Sleep_Efficiency.csv')
atributos = ['Age', 'Sleep duration','REM sleep percentage','Deep sleep percentage','Light sleep percentage','Awakenings', 'Caffeine consumption', 'Alcohol consumption', 'Exercise frequency','sex','smoking','Bedhour','Wakehour']
percentual_teste = 0.2

# Código

# Carga
dataset = carregador.carregar_dados(url_dados, atributos)
# Pré-processamento
dataset,X_train, X_test, Y_train, Y_test = pre_processador.pre_processar(dataset,
                                                               percentual_teste)
dataset.head()

# Treinamento do modelo
model = modelo.treinar_SVM(X_train, Y_train)
# Impressão do resultado da avaliação
print(avaliador.avaliar_acuracia(model, X_test, Y_test))

import pickle

pickle_out  = open('modelo_treinado.pkl','wb')
pickle.dump(model,pickle_out)
pickle_out.close()

0.6282051282051282


In [66]:
dataset.head()

,Age,Sleep duration,REM sleep percentage,Deep sleep percentage,Light sleep percentage,Awakenings,Caffeine consumption,Alcohol consumption,Exercise frequency,sex,smoking,Bedhour,Wakehour,categoric_sleep_eff
0,65,6.0,18,70,12,0.0,0.0,0.0,3.0,0,1,1,7,Otimo
1,69,7.0,19,28,53,3.0,0.0,3.0,3.0,1,1,2,9,Ruim
2,40,8.0,20,70,10,1.0,0.0,0.0,3.0,0,0,21,5,Otimo
3,40,6.0,23,25,52,3.0,50.0,5.0,1.0,0,1,2,8,Pessimo
4,57,8.0,27,55,18,3.0,0.0,3.0,3.0,1,0,1,9,Bom
